# **Entregable 3 - Integración de Modelos NER y Detección de Negación/Incertidumbre para la Extracción Estructurada de Entidades en Historias Clínicas de Cáncer de Mama**

**Integrantes:**
- Yeraldin Tafur
- Mayra Erazo
- Roberto Ceballos
- Katheryn Sanchez
<p align="center">
    <img src="https://innovacioneducativa.upc.edu.pe/wp-content/uploads/2025/04/dia-mundial-de-la-salud-1170x532.jpg" width="800">
</p>


# Introducción

En esta entrega se presenta un sistema automático para la extracción estructurada de información clínica relacionada con el cáncer de mama.

El sistema integra dos modelos preentrenados de procesamiento de lenguaje natural (PLN):  
- Un modelo de **reconocimiento de entidades nombradas (NER)**.  
- Un modelo de **detección de negación e incertidumbre**.

A partir de historias clínicas en formato texto, se identifican entidades biomédicas relevantes y se clasifica su estado como:
- **Afirmativa**
- **Negada**
- **Sospechosa**

Como resultado, se genera un archivo **CSV estructurado** que facilita el análisis clínico y la toma de decisiones basada en datos.


In [ ]:
# ============ Bloque 1: Instalación de dependencias y autenticación ============

# Instalación de las librerías necesarias desde Hugging Face
!pip install transformers[torch]
!pip install accelerate

# Instalación de tqdm para mostrar barras de progreso
!pip install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
# ============ Bloque 2: Autenticación con Hugging Face y montaje de Google Drive ============

from huggingface_hub import login
login(getpass("Introduce tu token de Hugging Face: ")) # Token de acceso personal a Hugging Face...

In [ ]:
# Montar Google Drive para acceder a archivos almacenados
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# ============ Bloque 3: Carga de historias clínicas desde archivos .txt ============

from google.colab import files

# Permite al usuario subir archivos .txt desde su máquina local
uploaded = files.upload()

# Ver nombres de los archivos subidos
all_frases_df = []
for file_name in uploaded.keys():
    print(f"Archivo subido: {file_name}")

    # Leer el contenido del archivo
    with open(file_name, 'r', encoding='utf-8') as file:
        lines = [line.strip() for line in file if line.strip()]
        all_frases_df.extend(lines)

print(f"Total oraciones cargadas: {len(all_frases_df)}")
print("Ejemplo de oración:", all_frases_df[0])

Saving 54.txt to 54.txt
Saving 55.txt to 55.txt
Saving 69.txt to 69.txt
Saving 100(1).txt to 100(1).txt
Saving 100.txt to 100.txt
Saving 101.txt to 101.txt
Saving 120.txt to 120.txt
Saving 121.txt to 121.txt
Saving 122.txt to 122.txt
Saving 123.txt to 123.txt
Saving 124.txt to 124.txt
Saving 125.txt to 125.txt
Saving 126.txt to 126.txt
Saving 127.txt to 127.txt
Saving 128.txt to 128.txt
Saving 129.txt to 129.txt
Saving 130.txt to 130.txt
Saving 130m.txt to 130m.txt
Saving 131.txt to 131.txt
Saving 132.txt to 132.txt
Saving 133.txt to 133.txt
Saving 134.txt to 134.txt
Saving 135.txt to 135.txt
Saving 136.txt to 136.txt
Saving 137.txt to 137.txt
Saving 138.txt to 138.txt
Saving 139.txt to 139.txt
Saving 140.txt to 140.txt
Saving 141.txt to 141.txt
Saving 142.txt to 142.txt
Saving 143.txt to 143.txt
Saving 144.txt to 144.txt
Saving 145.txt to 145.txt
Saving 146.txt to 146.txt
Saving 147.txt to 147.txt
Saving 148.txt to 148.txt
Saving 149.txt to 149.txt
Saving 150.txt to 150.txt
Saving 151

Cargamos modelo de cancer de mama

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
from tqdm import tqdm

# ============ Bloque 4: Diccionarios de etiquetas ============

# Diccionario que mapea los índices a etiquetas del modelo NER biomédico
id2label_ner = {
    0: "B-AGE", 1: "B-STAGE", 2: "B-DATE", 3: "B-IMPLICIT_DATE", 4: "B-TNM",
    5: "B-FAMILY", 6: "B-OCURRENCE_EVENT", 7: "B-TOXIC_HABITS", 8: "B-HABIT-QUANTITY",
    9: "B-TREATMENT_NAME", 10: "B-LINE_CICLE_NUMBER", 11: "B-SURGERY", 12: "B-DRUG",
    13: "B-DOSE", 14: "B-FREQ", 15: "B-BIOMARKER", 16: "B-CLINICAL_SERVICE",
    17: "B-COMORBIDITY", 18: "B-PROGRESION", 19: "B-GINECOLOGICAL_HISTORY",
    20: "B-GINE_OBSTETRICS", 21: "B-ALLERGIES", 22: "B-DURATION",
    23: "I-AGE", 24: "I-STAGE", 25: "I-DATE", 26: "I-IMPLICIT_DATE",
    27: "I-TNM", 28: "I-FAMILY", 29: "I-OCURRENCE_EVENT", 30: "I-TOXIC_HABITS",
    31: "I-HABIT-QUANTITY", 32: "I-TREATMENT_NAME", 33: "I-LINE_CICLE_NUMBER",
    34: "I-SURGERY", 35: "I-DRUG", 36: "I-DOSE", 37: "I-FREQ", 38: "I-BIOMARKER",
    39: "I-CLINICAL_SERVICE", 40: "I-COMORBIDITY", 41: "I-PROGRESION",
    42: "I-GINECOLOGICAL_HISTORY", 43: "I-GINE_OBSTETRICS", 44: "I-ALLERGIES",
    45: "I-DURATION", 46: "B-CANCER_CONCEPT", 47: "I-CANCER_CONCEPT", 48: "O"
}
label2id_ner = {v: k for k, v in id2label_ner.items()}

# Diccionario para las etiquetas del modelo de Negación/Incertidumbre
id2label_neg = {
    0: "B-NEG", 1: "B-NSCO", 2: "B-UNC", 3: "B-USCO", 4: "I-NEG",
    5: "I-NSCO", 6: "I-UNC", 7: "I-USCO", 8: "O"
}
label2id_neg = {v: k for k, v in id2label_neg.items()}

# ============ Bloque 5: Carga de modelos y tokenizadores ============

# Cargar el modelo NER desde Hugging Face
model_ner = AutoModelForTokenClassification.from_pretrained(
    "anvorja/breast-cancer-biomedical-ner-sp-1",
    id2label=id2label_ner,
    label2id=label2id_ner
)
tokenizer_ner = AutoTokenizer.from_pretrained("anvorja/breast-cancer-biomedical-ner-sp-1", use_fast=True)

# Cargar el modelo de detección de negación/uncertainty
model_neg = AutoModelForTokenClassification.from_pretrained(
    "JuanSolarte99/bert-base-uncased-finetuned-ner-negation_detection_NUBES",
    id2label=id2label_neg,
    label2id=label2id_neg
)
tokenizer_neg = AutoTokenizer.from_pretrained("JuanSolarte99/bert-base-uncased-finetuned-ner-negation_detection_NUBES", use_fast=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_ner.to(device)
model_neg.to(device)

# ============ Bloque 6: Determinación del estado de las entidades (Negada, Sospechosa, Afirmativa) ============

resultados = []

for idx, sentence in enumerate(tqdm(all_frases_df)):
    enc_ner = tokenizer_ner(sentence, return_tensors="pt", truncation=True, max_length=512).to(device)
    enc_neg = tokenizer_neg(sentence, return_tensors="pt", truncation=True, max_length=512).to(device)

    with torch.no_grad():
        logits_ner = model_ner(**enc_ner).logits
        logits_neg = model_neg(**enc_neg).logits

    pred_ner = torch.argmax(logits_ner, dim=-1)[0].tolist()
    pred_neg = torch.argmax(logits_neg, dim=-1)[0].tolist()

    tokens_ner = tokenizer_ner.convert_ids_to_tokens(enc_ner["input_ids"][0])
    word_ids_ner = enc_ner.word_ids(batch_index=0)
    word_ids_neg = enc_neg.word_ids(batch_index=0)

    entidades = []
    entidad_actual = ""
    etiqueta_actual = ""
    indices_entidad = []

    for i, (token, label_id, word_id) in enumerate(zip(tokens_ner, pred_ner, word_ids_ner)):
        if word_id is None:
            continue
        label = id2label_ner[label_id]
        if label.startswith("B-"):
            if entidad_actual:
                entidades.append((entidad_actual.strip(), etiqueta_actual, indices_entidad))
            entidad_actual = token.replace("▁", "").replace("##", "")
            etiqueta_actual = label[2:]
            indices_entidad = [i]
        elif label.startswith("I-") and etiqueta_actual == label[2:]:
            entidad_actual += " " + token.replace("▁", "").replace("##", "")
            indices_entidad.append(i)
        else:
            if entidad_actual:
                entidades.append((entidad_actual.strip(), etiqueta_actual, indices_entidad))
            entidad_actual = ""
            etiqueta_actual = ""
            indices_entidad = []

    if entidad_actual:
        entidades.append((entidad_actual.strip(), etiqueta_actual, indices_entidad))

    for entidad, etiqueta, indices in entidades:
        estados_detectados = set()
        for i in indices:
            if i < len(pred_neg):
                etiqueta_neg = id2label_neg.get(pred_neg[i], "O")
                if "NEG" in etiqueta_neg:
                    estados_detectados.add("Negada")
                elif "UNC" in etiqueta_neg or "USCO" in etiqueta_neg:
                    estados_detectados.add("Sospechosa")

        if "Negada" in estados_detectados:
            estado_final = "Negada"
        elif "Sospechosa" in estados_detectados:
            estado_final = "Sospechosa"
        else:
            estado_final = "Afirmativa"

        resultados.append({
            "patient_id": idx,
            "sentence": sentence,
            "NER": etiqueta,
            "Estado": estado_final
        })

# ============ Bloque 7: Construcción del DataFrame y exportación a CSV ============

df_resultado = pd.DataFrame(resultados)
df_resultado.to_csv("resultado_entidades.csv", index=False)
print("Archivo CSV generado con éxito.")


100%|██████████| 1310/1310 [00:39<00:00, 32.88it/s]

Archivo CSV generado con éxito.
